In [ ]:
%reset -f

Запуск команд через командний рядок прямо з блокнота

In [ ]:
!pwd

/content


In [ ]:
!cd sample_data/ ; ls

anscombe.json		     california_housing_train.csv  mnist_train_small.csv
california_housing_test.csv  mnist_test.csv		   README.md


In [ ]:
!ls

prog.py  sample_data


Командою, яка запускається, може бути також і інтерпретатор python-а, якому для виконання ми передаємо програму з файла `prog.py`

In [ ]:
!python3 prog.py 12.0 125 "New text string"

Program has started, with ['prog.py', '12.0', '125', 'New text string']
Program has almost finished
Traceback (most recent call last):
  File "/content/prog.py", line 7, in <module>
    raise ValueError
ValueError


Файл `prog.py` - імітує довготривалі обчислення тривалістю 1.5 секунди, друкує повідомлення на екран та викликає помилку

```python
import time
import sys

print('Program has started, with', sys.argv)
time.sleep(1.5)
print('Program has almost finished')
raise ValueError
print('Program has finally finished')
```
(має бути створений окремо, у поточному каталозі)

В усіх прикладах виконання команд призводило до створення нового процеса, але до цього часу наша програма (якби ми писали її в одній з комірок - не плутати із `prog.py`) не мала контролю над процесом, який створювався.

Запускатимемо тепер довільні "зовнішні" процеси у контрольований спосіб

In [ ]:
import subprocess

In [ ]:
p = ["ls"]
process = subprocess.Popen(p, stdout=subprocess.PIPE, text=True)
so, se = process.communicate()
print('The process output is:', so)

The process output is: prog.py
sample_data



In [ ]:
so

'prog.py\nsample_data\n'

In [ ]:
p = ["python3", "prog.py", "125"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           text=True)
so, se = process.communicate()
print('The process output is:', so)

The process output is: Program has started, with ['prog.py', '125']
Program has almost finished



In [ ]:
if len(se)==0:
  print('Помилок не було')
else:
  print('Помилки були!')
  print(se)


Помилки були!
Traceback (most recent call last):
  File "/content/prog.py", line 7, in <module>
    raise ValueError
ValueError



Файл `prog2.py` - "зовнішня програма", яка тепер вимагає ще і введення від користувача

```python
import time

x = int(input('Введіть x'))
time.sleep(1.5)
print(f'Результат: {x+1}')
```
(має бути створений окремо, у поточному каталозі)

In [ ]:
p = ["python3", "prog2.py"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE, # !!!
                           text=True)
so, se = process.communicate(input='128')
print('The process output is:', so)
print('Errors are:', se)

The process output is: Введіть xРезультат: 129

Errors are: 


Файл `prog3.py` - вимагає введень *двох* значень від користувача

```python
import time

x = int(input('Введіть x'))
time.sleep(1.0)
print(f'Результат: {x+1}')

y = int(input('Введіть y'))
time.sleep(1.0)
print(f'Результат #2: {y**2}')
```
(має бути створений окремо, у поточному каталозі)

In [ ]:
import time

p = ["python3", "prog3.py"]
t_start = time.time()
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)
so, se = process.communicate(input='128\n56') # !!!: \n
t_stop = time.time()

print('The process output is:', so)
print('Errors are:', se)
print('Execution time:', t_stop - t_start)

The process output is: Введіть xРезультат: 129
Введіть yРезультат #2: 3136

Errors are: 
Execution time: 2.041544198989868


Файл `prog4.py` - вимагає кількох введень від користувача, причому те, скільки запитів на введення буде не є відомим завчасно

```python
import time

for i in range(10):
  s = input('Введіть x')
  time.sleep(2.0)
  try:
    x = int(s)
    break
  except:
    print('Некоректний ввід')


time.sleep(2.0)
print(f'Результат: {x+1}')

```
(має бути створений окремо, у поточному каталозі)

In [ ]:
import time

p = ["python3", "prog4.py"]
t_start = time.time()
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)
so, se = process.communicate(input='jyvkuyvu\nukiyviy') # !!!: \n
t_stop = time.time()

print('The process output is:', so)
print('Errors are:', se)
print('Execution time:', t_stop - t_start)

The process output is: Введіть xНекоректний ввід
Введіть xНекоректний ввід
Введіть x
Errors are: Traceback (most recent call last):
  File "/content/prog4.py", line 4, in <module>
    s = input('Введіть x')
        ^^^^^^^^^^^^^^^^^^
EOFError: EOF when reading a line

Execution time: 0.03753089904785156


Приклад "блокуючої операції" -- цикли зі зчитування із `process.stdout` та `process.stdout` тривають аж поки "дочірній" процес `python3 prog.py` не завершиться, - хоча `print('waiting...')` *в нашій програмі* було почало виконуватися не чекаючи завершення "дочірнього" процесу

In [ ]:
p = ["python3", "prog.py"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)
print('waiting...')
for line in process.stdout:
  print('OUTPUT:', line)

for line in process.stderr:
  print('ERR:', line)


waiting...
OUTPUT: Program has started, with ['prog.py']

OUTPUT: Program has almost finished

ERR: Traceback (most recent call last):

ERR:   File "/content/prog.py", line 7, in <module>

ERR:     raise ValueError

ERR: ValueError



Потоки `threading.Thread` допомагають "розпаралелити очікування", не блокуючи роботу основної програми

In [ ]:
import threading

def read_std_out(pipe):
    for line in pipe:
        print("На екрані:", line)

def read_std_err(pipe):
    for line in pipe:
        print("В помилках", line)

p = ["python3", "prog.py"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)


stdout_thread = threading.Thread(target=read_std_out, args=(process.stdout, ))
stderr_thread = threading.Thread(target=read_std_err, args=(process.stderr, ))
stdout_thread.start()
stderr_thread.start()

process.wait()

На екрані: Program has started, with ['prog.py']

На екрані: Program has almost finished

В помилках Traceback (most recent call last):

В помилках   File "/content/prog.py", line 7, in <module>

В помилках     raise ValueError

В помилках ValueError



1

Завдяки цьому основна програма може займатися, наприклад, направленням даних в "дочірній процес" так, ніби ці данні вводив би користувач

In [ ]:
import threading

def read_std_out(pipe):
    for line in pipe:
        print("На екрані:", line)

def read_std_err(pipe):
    for line in pipe:
        print("В помилках", line)

p = ["python3", "prog4.py"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)


stdout_thread = threading.Thread(target=read_std_out, args=(process.stdout, ))
stderr_thread = threading.Thread(target=read_std_err, args=(process.stderr, ))
stdout_thread.start()
stderr_thread.start()

print("Let's feed some data into the program...")

process.stdin.write('uvkyvky!\n')
process.stdin.flush()

process.stdin.write('12\n')
process.stdin.flush()

process.wait()



Let's feed some data into the program...
На екрані: Введіть xНекоректний ввід

На екрані: Введіть xРезультат: 13



0

Використання `Queue` із модуля `queue` для обміну даними між основною програмою і потоком

In [ ]:
from queue import Queue

def read_std_out(pipe, q):
    for line in pipe:
        q.put(line)

def read_std_err(pipe):
    for line in pipe:
        print("В помилках", line)

p = ["python3", "prog4.py"]
process = subprocess.Popen(p,
                           stdout=subprocess.PIPE,
                           stderr=subprocess.PIPE,
                           stdin=subprocess.PIPE,
                           text=True)


qo = Queue()
stdout_thread = threading.Thread(target=read_std_out, args=(process.stdout, qo))
stderr_thread = threading.Thread(target=read_std_err, args=(process.stderr, ))
stdout_thread.start()
stderr_thread.start()

print("Let's feed some data into the program...")

inps = ['ublubliu', 'ilubkyvkyuc', '123', 'lin;oibo']
for s in inps:
  print('спробуємо вветси', s)
  process.stdin.write(s + '\n')
  process.stdin.flush()
  while qo.empty():
    print('чекаємо...')
    time.sleep(0.3)
  po = qo.get()
  print('Процес відповів', po)
  if po.find('Некоректний') == -1:
    print('Ура! Нарешті вгадали!')
    break

process.wait()



Let's feed some data into the program...
спробуємо вветси ublubliu
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
Процес відповів Введіть xНекоректний ввід

спробуємо вветси ilubkyvkyuc
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
Процес відповів Введіть xНекоректний ввід

спробуємо вветси 123
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
чекаємо...
Процес відповів Введіть xРезультат: 124

Ура! Нарешті вгадали!


0

Типовий приклад, коли корисно "розпаралелити очікування" - скачування із Інтернету великої кількості файлів.

Підготуємо перелік посилань для скачування

In [ ]:
import requests
from bs4 import BeautifulSoup

# Download the webpage
url = 'https://en.wikipedia.org/wiki/Python_(programming_language)'
response = requests.get(url)
web_content = response.content

# Parse the webpage content
soup = BeautifulSoup(web_content, 'html.parser')

# Extract all links
links = [a.get('href') for a in soup.find_all('a', href=True)]

# Convert relative links to full URLs
base_url = 'https://en.wikipedia.org'
links = [link if link.startswith('http') else base_url + link for link in links]

# Print the links
for link in links:
    print(link)


https://en.wikipedia.org#bodyContent
https://en.wikipedia.org/wiki/Main_Page
https://en.wikipedia.org/wiki/Wikipedia:Contents
https://en.wikipedia.org/wiki/Portal:Current_events
https://en.wikipedia.org/wiki/Special:Random
https://en.wikipedia.org/wiki/Wikipedia:About
https://en.wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us
https://en.wikipedia.org/wiki/Help:Contents
https://en.wikipedia.org/wiki/Help:Introduction
https://en.wikipedia.org/wiki/Wikipedia:Community_portal
https://en.wikipedia.org/wiki/Special:RecentChanges
https://en.wikipedia.org/wiki/Wikipedia:File_upload_wizard
https://en.wikipedia.org/wiki/Main_Page
https://en.wikipedia.org/wiki/Special:Search
https://donate.wikimedia.org/?wmf_source=donate&wmf_medium=sidebar&wmf_campaign=en.wikipedia.org&uselang=en
https://en.wikipedia.org/w/index.php?title=Special:CreateAccount&returnto=Python+%28programming+language%29
https://en.wikipedia.org/w/index.php?title=Special:UserLogin&returnto=Python+%28programming+language%

Для скачування за підготовленими посиланнями використаємо такий спосіб:

In [ ]:
%%time
import requests
x = requests.get('http://example.com/file.txt').content
print(x)


b'<!doctype html>\n<html>\n<head>\n    <title>Example Domain</title>\n\n    <meta charset="utf-8" />\n    <meta http-equiv="Content-type" content="text/html; charset=utf-8" />\n    <meta name="viewport" content="width=device-width, initial-scale=1" />\n    <style type="text/css">\n    body {\n        background-color: #f0f0f2;\n        margin: 0;\n        padding: 0;\n        font-family: -apple-system, system-ui, BlinkMacSystemFont, "Segoe UI", "Open Sans", "Helvetica Neue", Helvetica, Arial, sans-serif;\n        \n    }\n    div {\n        width: 600px;\n        margin: 5em auto;\n        padding: 2em;\n        background-color: #fdfdff;\n        border-radius: 0.5em;\n        box-shadow: 2px 3px 7px 2px rgba(0,0,0,0.02);\n    }\n    a:link, a:visited {\n        color: #38488f;\n        text-decoration: none;\n    }\n    @media (max-width: 700px) {\n        div {\n            margin: 0 auto;\n            width: auto;\n        }\n    }\n    </style>    \n</head>\n\n<body>\n<div>\n    

Заміряємо час скачування 20 посилань одне за однм (послідовно)

In [ ]:
%%time
for i, u in enumerate(links[:20]):
    x = requests.get(u).content
    print (x[110:210], u)


b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' https://en.wikipedia.org#bodyContent
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' https://en.wikipedia.org/wiki/Main_Page
b'main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-dis' https://en.wikipedia.org/wiki/Wikipedia:Contents
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' https://en.wikipedia.org/wiki/Portal:Current_events
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' https://en.wikipedia.org/wiki/Special:Random
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' https://en.wikipedia.org/wiki/Wikipedia:About
b' icon" href="/favicon.ico">\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; color: #20' 

І заміряємо аналогічний час, коли скачуванням кожного посилання займається окремий потік

In [ ]:
%%time

def func(url, i):
    x = requests.get(url).content
    print (x[110:210], i, url)

threads = []
for i, u in enumerate(links[:20]):
    th = threading.Thread(target=func, args=(u,i))
    th.start()
    threads.append(th)
for th in threads:
    th.join()



b'main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-dis' 2 https://en.wikipedia.org/wiki/Wikipedia:Contents
b' icon" href="/favicon.ico">\n<style>\n* { margin: 0; padding: 0; }\nbody { background: #fff; color: #20' 6 https://en.wikipedia.org//en.wikipedia.org/wiki/Wikipedia:Contact_us
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' 7 https://en.wikipedia.org/wiki/Help:Contents
b'main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-dis' 8 https://en.wikipedia.org/wiki/Help:Introduction
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' 5 https://en.wikipedia.org/wiki/Wikipedia:About
b'main-page-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-client' 1 https://en.wikipedia.org/wiki/Main_Page
b'main-page-header-disabled vector-feature-page-tools-pinned-disa